In [ ]:
!pip install transformers peft datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL_NAME = "Qwen/Qwen3-0.6B"

base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0,
    bias="none"
)

lora_model = get_peft_model(base_model, lora_config)

In [ ]:
from datasets import load_dataset

ds = load_dataset("mhhmm/typescript-instruct-20k", split="train")

def preprocess(examples):
    inputs = examples["instruction"]
    targets = [out + base_tokenizer.eos_token for out in examples["output"]]
    model_inputs = base_tokenizer(inputs, truncation=True, padding="max_length", max_length=2048)
    labels = base_tokenizer(targets, truncation=True, padding="max_length", max_length=2048).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_ds = ds.map(preprocess, batched=True, remove_columns=ds.column_names)

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/35.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
import os
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

class SaveEveryNEpochCallback(TrainerCallback):
    def __init__(self, save_every: int = 1):
        self.save_every = save_every

    def on_epoch_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        epoch = int(state.epoch)
        if epoch % self.save_every == 0:
            out_dir = os.path.join(args.output_dir, f"checkpoint-epoch{epoch}")
            os.makedirs(out_dir, exist_ok=True)
            model = kwargs["model"]
            model.save_pretrained(out_dir)
            print(f">>> Saved LoRA adapter model to {out_dir}")
        return control

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="qwen3-ts",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=25,
    save_total_limit=2,
    save_steps=200,
    report_to="none"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=base_tokenizer,
    callbacks = [SaveEveryNEpochCallback(save_every=1)],
)

trainer.train()

<ipython-input-6-812c4a2951e7>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
25,2.534900
50,1.504000
75,1.457700
100,1.509800
125,1.386200
150,1.390700
175,1.372300
200,1.466000
225,1.389600
250,1.424500


>>> Saved LoRA adapter model to qwen3-ts/checkpoint-epoch1
>>> Saved LoRA adapter model to qwen3-ts/checkpoint-epoch2
>>> Saved LoRA adapter model to qwen3-ts/checkpoint-epoch3


TrainOutput(global_step=1875, training_loss=1.3441496520996095, metrics={'train_runtime': 10932.1197, 'train_samples_per_second': 5.488, 'train_steps_per_second': 0.172, 'total_flos': 3.321888768e+17, 'train_loss': 1.3441496520996095, 'epoch': 3.0})